In [1]:
import torch
from transformers import pipeline
from datasets import load_dataset
from IPython.display import Audio, display

print("Версия PyTorch:", torch.__version__)
print("CUDA доступна:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Имя GPU:", torch.cuda.get_device_name(0))
else:
    print("Будет использоваться CPU (работать будет медленнее).")

c:\Users\andre\Desktop\KGEU\ProgramEng\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Версия PyTorch: 2.7.1+cu118
CUDA доступна: True
Имя GPU: NVIDIA GeForce RTX 4070 Ti


In [2]:
import os
os.environ["HF_DATASETS_AUDIO_ALLOW_TORCHCODEC"] = "0"

from datasets import load_dataset, Audio

# Загружаем dummy датасет
librispeech = load_dataset(
    "hf-internal-testing/librispeech_asr_dummy",
    split="validation[:1]"
)

# Приводим audio в режим raw bytes
librispeech = librispeech.cast_column("audio", Audio(decode=False))

sample = librispeech[0]

audio_bytes = sample["audio"]["bytes"]
print("Audio structure:", sample["audio"])

print("Оригинальный текст:")
print(sample["text"])


Audio structure: {'bytes': b'fLaC\x00\x00\x00"\x10\x00\x10\x00\x00\x08:\x00\x18x\x03\xe8\x00\xf0\x00\x01m\xf0\x84\xd1,\x1b\xc02I3e\xbe\x02\xd0\x98\xcf\t\xc4\x03\x00\x00\x12\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x10\x00\x04\x00\x00( \x00\x00\x00reference libFLAC 1.2.1 20070917\x00\x00\x00\x00\x81\x00 \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

In [3]:
from transformers import pipeline
import tempfile

asr = pipeline("automatic-speech-recognition", model="openai/whisper-small")

# создаём временный файл
with tempfile.NamedTemporaryFile(suffix=".flac", delete=False) as f:
    f.write(audio_bytes)
    temp_path = f.name

result = asr(temp_path)

print("ТРАНСКРИПЦИЯ ОТ WHISPER:")
print(result["text"])

c:\Users\andre\Desktop\KGEU\ProgramEng\.venv\Lib\site-packages\huggingface_hub\file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


ТРАНСКРИПЦИЯ ОТ WHISPER:
 Mr. Quilter is the apostle of the middle classes, and we are glad to welcome his gospel.


In [6]:
from transformers import pipeline

# Маленькая быстрая модель суммаризации
summarizer = pipeline(
    "summarization",
    model="sshleifer/distilbart-cnn-12-6"
)

summary = summarizer(
    result["text"],     # текст от Whisper
    max_length=15,      
    min_length=5,
    do_sample=False
)

print("СУММАРИЗАЦИЯ:")
print(summary[0]["summary_text"])

СУММАРИЗАЦИЯ:
 Mr. Quilter is the apostle of the middle classes
